## ONNX Runtime Inference

In [6]:
import torch
import onnx
import onnxruntime as ort
import numpy as np
from rich import print
from rich.console import Console
import time
import onnx_graphsurgeon as gs

# Load the ONNX model
model = onnx.load("simple_bev_optimized.onnx")

# Check the model
onnx.checker.check_model(model)

print("Model is checked!")

ort_session = ort.InferenceSession(model.SerializeToString())

print([(x.name,x.shape) for x in ort_session.get_inputs()])
print([(x.name,x.shape) for x in ort_session.get_outputs()])


Model is checked!

[('rgb_camXs', [1, 6, 3, 224, 400]), ('pix_T_cams', [1, 6, 4, 4]), ('cam0_T_camXs', [1, 6, 4, 4])]

[
    ('output_0', [1, 128, 200, 200]),
    ('feat_bev_e', [1, 128, 200, 200]),
    ('seg_bev_e', [1, 1, 200, 200]),
    ('center_bev_e', [1, 1, 200, 200]),
    ('offset_bev_e', [1, 2, 200, 200])
]

In [8]:
graph=gs.import_onnx(model)

graph.inputs

[Variable (rgb_camXs): (shape=[1, 6, 3, 224, 400], dtype=float32),
 Variable (pix_T_cams): (shape=[1, 6, 4, 4], dtype=float32),
 Variable (cam0_T_camXs): (shape=[1, 6, 4, 4], dtype=float32)]

In [9]:
graph.cleanup().toposort()


['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_display_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_repr_mimebundle_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_repr_html_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_repr_markdown_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_repr_svg_' was not registered for opset 20. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 20. 
['\x1b'][E] Function: '_repr_png_' was no

Graph main_graph (Opset 20)
Local Functions: []
Inputs: [Variable (rgb_camXs): (shape=[1, 6, 3, 224, 400], dtype=float32), Variable (pix_T_cams): (shape=[1, 6, 4, 4], dtype=float32), Variable (cam0_T_camXs): (shape=[1, 6, 4, 4], dtype=float32)]
Nodes: Identity_348 (Identity)
	Inputs: [
		Constant (onnx::Conv_2477): (shape=[256], dtype=float32)
	]
	Outputs: [
		Variable (onnx::Conv_2489): (shape=None, dtype=None)
	]
Identity_349 (Identity)
	Inputs: [
		Constant (onnx::Conv_2479): (shape=[256, 256, 3, 3], dtype=float32)
	]
	Outputs: [
		Variable (onnx::Conv_2488): (shape=None, dtype=None)
	]
Identity_350 (Identity)
	Inputs: [
		Constant (onnx::Conv_2477): (shape=[256], dtype=float32)
	]
	Outputs: [
		Variable (onnx::Conv_2486): (shape=None, dtype=None)
	]
Identity_351 (Identity)
	Inputs: [
		Constant (onnx::Conv_2477): (shape=[256], dtype=float32)
	]
	Outputs: [
		Variable (onnx::Conv_2483): (shape=None, dtype=None)
	]
Identity_352 (Identity)
	Inputs: [
		Constant (onnx::Conv_2477): (sha

In [10]:
cleaned_model=gs.export_onnx(graph)
onnx.checker.check_model(cleaned_model)
onnx.save(cleaned_model,"cleaned.onnx")
